<h3> Ik heb nu een Datawarehouse database met tabellen en specifieke attributen om data mee goed te kunnen analyseren, nu moeten veel tabellen gemanipuleerd worden zie hier:

<h2> SQL Server connectie maken met SourceDataModel

In [2]:
import pyodbc
import pandas as pd

# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database = 'GreatOutdoorsSDM'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()



<h4> inlezen alle data van SourceDataModel

In [3]:
# Haal alle tabellen op
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = [row.TABLE_NAME for row in cursor.fetchall()]

# Dictionary om alle dataframes op te slaan
dataframes = {}

# Loop door alle tabellen en laad ze in Pandas DataFrames
for table in tables:
    query = f"SELECT * FROM {table}"
    df = pd.read_sql(query, conn)
    dataframes[table] = df
    print(f"Tabel '{table}' ingelezen met {df.shape[0]} rijen en {df.shape[1]} kolommen.")

# Sluit de verbinding
conn.close()

# Print de kolomnamen en de eerste paar rijen van elke DataFrame
for table_name, df in dataframes.items():
    print(f"\nTable: {table_name}")
    print("Columns:", df.columns.tolist())
    



Tabel 'age_group' ingelezen met 6 rijen en 3 kolommen.
Tabel 'sales_territory' ingelezen met 5 rijen en 2 kolommen.
Tabel 'country' ingelezen met 21 rijen en 6 kolommen.
Tabel 'retailer_segment' ingelezen met 12 rijen en 4 kolommen.
Tabel 'retailer_type' ingelezen met 8 rijen en 2 kolommen.
Tabel 'retailer_headquarters' ingelezen met 414 rijen en 11 kolommen.
Tabel 'retailer' ingelezen met 109 rijen en 4 kolommen.
Tabel 'retailer_site' ingelezen met 391 rijen en 9 kolommen.
Tabel 'retailer_contact' ingelezen met 391 rijen en 9 kolommen.
Tabel 'sales_demographic' ingelezen met 2484 rijen en 4 kolommen.
Tabel 'order_method' ingelezen met 7 rijen en 2 kolommen.
Tabel 'product_line' ingelezen met 5 rijen en 2 kolommen.
Tabel 'return_reason' ingelezen met 5 rijen en 2 kolommen.
Tabel 'sales_branch' ingelezen met 28 rijen en 7 kolommen.
Tabel 'product_type' ingelezen met 21 rijen en 3 kolommen.
Tabel 'product' ingelezen met 115 rijen en 9 kolommen.
Tabel 'sales_staff' ingelezen met 102 rijen

/var/folders/8f/wsxf5dvj0d31pnwb5zm1pkjr0000gn/T/ipykernel_3025/3291603298.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


<h2> manipulaties maken van tabellen

<h4> alle relevante tabellen uit het sourcedatamodel halen

In [4]:
# Haal de benodigde DataFrames uit de dictionary
product_df            = dataframes['product']
order_header_df       = dataframes['order_header']
order_details_df      = dataframes['order_details']
inventory_levels_df   = dataframes['inventory_levels']
product_forecast_df   = dataframes['product_forecast']
product_line_df       = dataframes['product_line']
product_type_df       = dataframes['product_type']
retailer_df           = dataframes['retailer']
retailer_hq_df        = dataframes['retailer_headquarters']
retailer_segment_df   = dataframes['retailer_segment']
retailer_type_df      = dataframes['retailer_type']
sales_branch_df       = dataframes['sales_branch']
sales_staff_df        = dataframes['sales_staff']
sales_territory_df    = dataframes['sales_territory']
order_method_df       = dataframes['order_method']
country_df            = dataframes['country']


<h2> manipuleren van de dataframes

<h3>1.	Fact_Sales (Omzetberekeningen) 
<h4>Bronnen: order_details, order_header, product 



In [5]:
# Merge order_details met order_header (op ORDER_NUMBER)
fact_sales = order_details_df.merge(order_header_df, on='ORDER_NUMBER', how='left', suffixes=('', '_hdr'))
# Merge met product (op PRODUCT_NUMBER)
fact_sales = fact_sales.merge(product_df, on='PRODUCT_NUMBER', how='left', suffixes=('', '_prod'))

# Bereken meetwaarden
fact_sales['Total_Sales'] = fact_sales['QUANTITY'] * fact_sales['UNIT_SALE_PRICE']
fact_sales['Profit'] = (fact_sales['UNIT_SALE_PRICE'] - fact_sales['UNIT_COST']) * fact_sales['QUANTITY']

# Creëer foreign key kolommen (placeholder mapping)
fact_sales['Product_Key'] = fact_sales['PRODUCT_NUMBER']
# Neem ORDER_DATE uit order_header en formatteer als tijd-key (bijv. YYYYMM)
fact_sales['Time_Key'] = pd.to_datetime(fact_sales['ORDER_DATE']).dt.strftime('%Y%m')
fact_sales['Sales_Team_Key'] = fact_sales['SALES_STAFF_CODE']   # Wordt gekoppeld via Sales Team-dimensie
fact_sales['Retailer_Key'] = fact_sales['RETAILER_SITE_CODE']   # Afgeleid; afhankelijk van de verdere koppeling
fact_sales['Order_Method_Key'] = fact_sales['ORDER_METHOD_CODE']
# Hernoem de kolom ORDER_DETAIL_CODE naar Sales_Key
fact_sales.rename(columns={'ORDER_DETAIL_CODE': 'Sales_Key'}, inplace=True)

# Houd alleen de relevante kolommen voor het datawarehouse
fact_sales = fact_sales[['Sales_Key','Product_Key','Time_Key','Sales_Team_Key', 'Retailer_Key','Order_Method_Key','Total_Sales', 'Profit']]

print(fact_sales)

       Sales_Key  Product_Key Time_Key  Sales_Team_Key  Retailer_Key  \
0        47737.0          1.0   202208            44.0          21.0   
1        47738.0          1.0   202204            50.0           9.0   
2        47739.0          1.0   202204            50.0          19.0   
3        47740.0          1.0   202210            50.0           4.0   
4        47741.0          1.0   202204            43.0          21.0   
...          ...          ...      ...             ...           ...   
37752   115289.0        112.0   202401            96.0         335.0   
37753   115290.0        113.0   202409            96.0         335.0   
37754   115291.0        114.0   202403            96.0         335.0   
37755   115292.0        115.0   202401            96.0         335.0   
37756   115293.0        115.0   202409            96.0         335.0   

       Order_Method_Key  Total_Sales      Profit  
0                   7.0   553.560013  185.640003  
1                   7.0   830.340

<h3> 2. Fact_Inventory (Voorraadbeheer) 
<h4>Bronnen: inventory_levels, product (en product_forecast voor de Expected_Volume)


In [6]:
# Merge inventory_levels met product_forecast op PRODUCT_NUMBER, en matching jaar en maand
fact_inventory = inventory_levels_df.merge(
    product_forecast_df, 
    left_on=['PRODUCT_NUMBER', 'INVENTORY_YEAR', 'INVENTORY_MONTH'],
    right_on=['PRODUCT_NUMBER', 'YEAR', 'MONTH'], 
    how='left', 
    suffixes=('_inv', '_forecast')
)

fact_inventory['Inventory_Turnover_Rate'] = fact_inventory['EXPECTED_VOLUME'] / fact_inventory['INVENTORY_COUNT']
fact_inventory['Product_Key'] = fact_inventory['PRODUCT_NUMBER']
fact_inventory['Time_Key'] = fact_inventory['INVENTORY_YEAR'].astype(str) + fact_inventory['INVENTORY_MONTH'].astype(str).str.zfill(2)

# Voeg de Inventory_Key kolom toe
fact_inventory['Inventory_Key'] = fact_inventory.index

fact_inventory = fact_inventory[['Inventory_Key','Product_Key','Time_Key', 'INVENTORY_COUNT',  'Inventory_Turnover_Rate']]

print(fact_inventory)                   

      Inventory_Key  Product_Key   Time_Key  INVENTORY_COUNT  \
0                 0          1.0  2022.01.0           2007.0   
1                 1          2.0  2022.01.0           1718.0   
2                 2          3.0  2022.01.0           2100.0   
3                 3          4.0  2022.01.0           4914.0   
4                 4          5.0  2022.01.0            846.0   
...             ...          ...        ...              ...   
3538           3538        111.0  2024.09.0           1014.0   
3539           3539        112.0  2024.09.0           2108.0   
3540           3540        113.0  2024.09.0           5091.0   
3541           3541        114.0  2024.09.0            580.0   
3542           3542        115.0  2024.09.0           5634.0   

      Inventory_Turnover_Rate  
0                    0.261086  
1                    0.196158  
2                    0.167619  
3                    0.041107  
4                    0.198582  
...                       ...  
3538   

<h3>3. Fact_Forecast (Verkoopverwachtingen)
<h4> Bronnen: product_forecast, product

In [7]:
fact_forecast = product_forecast_df.merge(product_df, on='PRODUCT_NUMBER', how='left')
fact_forecast['Product_Key'] = fact_forecast['PRODUCT_NUMBER']
fact_forecast['Time_Key'] = fact_forecast['YEAR'].astype(str) + fact_forecast['MONTH'].astype(str).str.zfill(2)

# Voeg de Forecast_Key kolom toe
fact_forecast['Forecast_Key'] = fact_forecast.index
fact_forecast = fact_forecast[['Forecast_Key' , 'Product_Key', 'Time_Key','EXPECTED_VOLUME']]

print(fact_forecast)

      Forecast_Key  Product_Key   Time_Key  EXPECTED_VOLUME
0                0          1.0  2022.01.0            524.0
1                1          1.0  2022.02.0            403.0
2                2          1.0  2022.03.0            750.0
3                3          1.0  2022.04.0           1386.0
4                4          1.0  2022.05.0            544.0
...            ...          ...        ...              ...
3524          3524        115.0  2024.05.0            278.0
3525          3525        115.0  2024.06.0            561.0
3526          3526        115.0  2024.07.0            412.0
3527          3527        115.0  2024.08.0            282.0
3528          3528        115.0  2024.09.0            920.0

[3529 rows x 4 columns]


<h3>4. Dim_Product
<h4> Bronnen: product, product_type, product_line

In [8]:
 # Merge product met product_type
dim_product = product_df.merge(product_type_df, on='PRODUCT_TYPE_CODE', how='left', suffixes=('', '_ptype'))
# Merge met product_line op PRODUCT_LINE_CODE (uit product_type)
dim_product = dim_product.merge(product_line_df, on='PRODUCT_LINE_CODE', how='left', suffixes=('', '_pline'))

dim_product['Product_Key'] = dim_product['PRODUCT_NUMBER']
dim_product['Product_Name'] = dim_product['PRODUCT_NAME']
dim_product['Product_Type_EN'] = dim_product['PRODUCT_TYPE_EN']
dim_product['Product_Line_EN'] = dim_product['PRODUCT_LINE_EN']
dim_product['Production_Cost'] = dim_product['PRODUCTION_COST']
dim_product['Margin'] = dim_product['MARGIN']

dim_product = dim_product[['Product_Key', 'Product_Name', 'Product_Type_EN', 'Product_Line_EN', 'Production_Cost', 'Margin']]

print(dim_product)

     Product_Key                 Product_Name   Product_Type_EN  \
0            1.0          TrailChef Water Bag      Cooking Gear   
1            2.0            TrailChef Canteen      Cooking Gear   
2            3.0        TrailChef Kitchen Kit      Cooking Gear   
3            4.0                TrailChef Cup      Cooking Gear   
4            5.0           TrailChef Cook Set      Cooking Gear   
..           ...                          ...               ...   
110        111.0        Blue Steel Max Putter           Putters   
111        112.0  Course Pro Golf and Tee Set  Golf Accessories   
112        113.0          Course Pro Umbrella  Golf Accessories   
113        114.0          Course Pro Golf Bag  Golf Accessories   
114        115.0            Course Pro Gloves  Golf Accessories   

       Product_Line_EN  Production_Cost  Margin  
0    Camping Equipment         4.000000    0.33  
1    Camping Equipment         9.220000    0.23  
2    Camping Equipment        15.930000    0.

<h3>5. Dim_Time
<h4>

In [9]:
# Vul dim_time met een datumreeks op basis van de minimale en maximale datums uit Order_Header.
min_date = pd.to_datetime(order_header_df['ORDER_DATE']).min()
max_date = pd.to_datetime(order_header_df['ORDER_DATE']).max()
date_range = pd.date_range(start=min_date, end=max_date, freq='D')
dim_time = pd.DataFrame({
    'Time_Key': date_range.strftime('%Y%m%d'),
    'Year': date_range.year,
    'Month': date_range.month,
    'Day': date_range.day,
    'Quarter': date_range.quarter,
    'Weekday': date_range.day_name()
})
print(dim_time)

      Time_Key  Year  Month  Day  Quarter    Weekday
0     20220102  2022      1    2        1     Sunday
1     20220103  2022      1    3        1     Monday
2     20220104  2022      1    4        1    Tuesday
3     20220105  2022      1    5        1  Wednesday
4     20220106  2022      1    6        1   Thursday
...        ...   ...    ...  ...      ...        ...
998   20240926  2024      9   26        3   Thursday
999   20240927  2024      9   27        3     Friday
1000  20240928  2024      9   28        3   Saturday
1001  20240929  2024      9   29        3     Sunday
1002  20240930  2024      9   30        3     Monday

[1003 rows x 6 columns]


<h3>6. Dim_Retailer
<h4>Bronnen: retailer, retailer_headquarters, retailer_type, retailer_segment, (en country voor Country_EN)

In [10]:
dim_retailer = retailer_df.merge(retailer_hq_df, on='RETAILER_CODEMR', how='left', suffixes=('', '_hq'))
dim_retailer = dim_retailer.merge(retailer_type_df, on='RETAILER_TYPE_CODE', how='left', suffixes=('', '_rtype'))
dim_retailer = dim_retailer.merge(retailer_segment_df, on='SEGMENT_CODE', how='left', suffixes=('', '_seg'))
dim_retailer = dim_retailer.merge(country_df[['COUNTRY_CODE', 'COUNTRY_EN']], on='COUNTRY_CODE', how='left')
dim_retailer['Retailer_Key'] = dim_retailer['RETAILER_CODE']
dim_retailer['Retailer_Name'] = dim_retailer['COMPANY_NAME']
dim_retailer['Retailer_Type_EN'] = dim_retailer['RETAILER_TYPE_EN']
dim_retailer['Segment_Name'] = dim_retailer['SEGMENT_NAME']
dim_retailer['Country_EN'] = dim_retailer['COUNTRY_EN']
dim_retailer['City'] = dim_retailer['CITY']      # uit retailer_headquarters
dim_retailer['Region'] = dim_retailer['REGION']  # uit retailer_headquarters
dim_retailer = dim_retailer[['Retailer_Key', 'Retailer_Name', 'Retailer_Type_EN', 'Segment_Name', 'Country_EN', 'City', 'Region']]

print(dim_retailer)

     Retailer_Key                 Retailer_Name        Retailer_Type_EN  \
0             1.0                     ActiForme  Equipment Rental Store   
1            15.0                    SportsClub               Golf Shop   
2            23.0                      Anapurna        Direct Marketing   
3            32.0             Cordages Discount         Warehouse Store   
4            33.0            Altitudes extrêmes           Outdoors Shop   
..            ...                           ...                     ...   
104         193.0                     Allo Allo        Direct Marketing   
105         194.0                  Caravanserai           Outdoors Shop   
106         209.0  Ausrüstungshaus Globetrotter        Department Store   
107         210.0               Golfer-Paradies               Golf Shop   
108         211.0   Outdoor-Fachgeschäft Müller           Outdoors Shop   

                  Segment_Name Country_EN            City Region  
0                          NaN  

<h3>7. Dim_Sales_Team
<h4>Bronnen: sales_staff, sales_branch


In [11]:
dim_sales_team = sales_staff_df.merge(sales_branch_df, on='SALES_BRANCH_CODE', how='left', suffixes=('', '_branch'))
dim_sales_team['Sales_Staff_Name'] = dim_sales_team['FIRST_NAME'].str.strip() + ' ' + dim_sales_team['LAST_NAME'].str.strip()
dim_sales_team['Sales_Team_Key'] = dim_sales_team['SALES_STAFF_CODE']
dim_sales_team['Position_EN'] = dim_sales_team['POSITION_EN']
dim_sales_team['Branch_City'] = dim_sales_team['CITY']  # uit sales_branch
# Self-join om Manager_Name te verkrijgen:
managers = sales_staff_df[['SALES_STAFF_CODE', 'FIRST_NAME', 'LAST_NAME']].rename(
    columns={'SALES_STAFF_CODE': 'MANAGER_CODE', 'FIRST_NAME': 'Manager_FirstName', 'LAST_NAME': 'Manager_LastName'}
)
dim_sales_team = dim_sales_team.merge(managers, on='MANAGER_CODE', how='left')
dim_sales_team['Manager_Name'] = dim_sales_team['Manager_FirstName'].str.strip() + ' ' + dim_sales_team['Manager_LastName'].str.strip()
dim_sales_team = dim_sales_team[['Sales_Team_Key', 'Sales_Staff_Name', 'Position_EN', 'Branch_City', 'Manager_Name']]

print(dim_sales_team)

     Sales_Team_Key   Sales_Staff_Name                   Position_EN  \
0               4.0         Denis Pagé                Branch Manager   
1               5.0   Élizabeth Michel  Level 3 Sales Representative   
2               6.0     Émile Clermont  Level 1 Sales Representative   
3               7.0     Étienne Jauvin  Level 2 Sales Representative   
4              12.0  Elsbeth Wiesinger  Level 2 Sales Representative   
..              ...                ...                           ...   
97            120.0     Giele Laermans  Level 1 Sales Representative   
98            121.0   François De Crée  Level 1 Sales Representative   
99            122.0     Yvette Lattrez  Level 3 Sales Representative   
100           123.0    Willi Seefelder  Level 2 Sales Representative   
101           124.0      Sabine Grüner  Level 3 Sales Representative   

    Branch_City   Manager_Name  
0         Paris     Jörg Kunze  
1         Paris     Denis Pagé  
2         Paris     Denis Pagé  
3  

<h3>8. Dim_Geography
<h4>Bronnen: country, sales_territory

In [12]:
dim_geography = country_df.merge(sales_territory_df, on='SALES_TERRITORY_CODE', how='left', suffixes=('', '_territory'))
dim_geography['Geography_Key'] = dim_geography['COUNTRY_CODE']
dim_geography['Country_EN'] = dim_geography['COUNTRY_EN']
dim_geography['Sales_Territory_Name'] = dim_geography['TERRITORY_NAME_EN']
dim_geography['Currency_Name'] = dim_geography['CURRENCY_NAME']
dim_geography = dim_geography[['Geography_Key', 'Country_EN', 'Sales_Territory_Name', 'Currency_Name']]

print(dim_geography)

    Geography_Key      Country_EN Sales_Territory_Name Currency_Name
0             1.0          France       Central Europe        francs
1             2.0         Germany       Central Europe         marks
2             3.0   United States             Americas       dollars
3             4.0          Canada             Americas       dollars
4             5.0         Austria      Southern Europe    schillings
5             6.0           Italy      Southern Europe          lira
6             7.0     Netherlands      Northern Europe      guilders
7             8.0     Switzerland       Central Europe        francs
8             9.0  United Kingdom       Central Europe        pounds
9            10.0          Sweden      Northern Europe         krona
10           11.0           Japan         Asia Pacific           yen
11           12.0          Taiwan         Asia Pacific    new dollar
12           13.0           Korea         Asia Pacific           won
13           14.0           China 

<h3>9. Dim_Order_Method --> Dim_Order_Method
<h4> Bronnen: Order_method

In [13]:
dim_order_method = order_method_df.copy()
dim_order_method = dim_order_method.rename(columns={'ORDER_METHOD_CODE': 'order_method_key'})
# Houd alleen de relevante attributen (bijv. Order_Method_Description)
dim_order_method = dim_order_method[['order_method_key', 'ORDER_METHOD_EN']]

print(dim_order_method)

   order_method_key ORDER_METHOD_EN
0               1.0             Fax
1               2.0       Telephone
2               3.0            Mail
3               4.0          E-mail
4               5.0             Web
5               7.0     Sales visit
6               8.0         Special


<h2>Alles in 1 dictionary zetten

In [14]:
dataWarehouse = {
    'fact_sales': fact_sales,
    'fact_inventory': fact_inventory,
    'fact_forecast': fact_forecast,
    'dim_product': dim_product,
    'dim_time': dim_time,
    'dim_retailer': dim_retailer,
    'dim_sales_team': dim_sales_team,
    'dim_geography': dim_geography,
    'dim_order_method': dim_order_method
}

<h2> connectie maken sql server van Data Warehouse

In [15]:
# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database = 'DataWarehouse'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

cursor.execute("SELECT @@VERSION;")
row = cursor.fetchone()
print("SQL Server versie:", row[0])


SQL Server versie: Microsoft Azure SQL Edge Developer (RTM) - 15.0.2000.1574 (ARM64) 
	Jan 25 2023 10:36:08 
	Copyright (C) 2019 Microsoft Corporation
	Linux (Ubuntu 18.04.6 LTS aarch64) <ARM64>


<h3> vullen van Data Warehouse

In [16]:
def clean_nan_values(dw):
    for table_name, df in dw.items():
        # Zet alle NaN naar None zodat SQL Server NULL kan verwerken
        dw[table_name] = df.astype(object).where(pd.notnull(df), None)
    return dw

# Pas toe op je dataWarehouse dictionary
dataWarehouse = clean_nan_values(dataWarehouse)


def upload_dataframes_to_sql(dw):
    try:
        with pyodbc.connect(connection_string, autocommit=False) as conn:
            cursor = conn.cursor()
            
            print("⏳ Uitschakelen van FOREIGN KEY constraints...")
            cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? NOCHECK CONSTRAINT ALL'")
            conn.commit()
            
            # Loop over elk tabel in de dw dictionary
            for table_name, df in dw.items():
                print(f"\nBezig met uploaden van tabel: {table_name}...")
                
                # Genereer kolomnamen en placeholders
                columns = ', '.join([f'[{col}]' for col in df.columns])
                placeholders = ', '.join(['?'] * len(df.columns))
                insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
                
                # Loop door elke rij in de DataFrame
                for idx, row in df.iterrows():
                    try:
                        row_data = tuple(row)
                        cursor.execute(insert_query, row_data)
                        conn.commit()  # commit na elke rij (kan je aanpassen voor batch commit)
                        print(f"  Rij {idx + 1} ✅ Upload voltooid")
                    except pyodbc.Error as e:
                        conn.rollback()  # rollback bij fout
                        error_msg = str(e).replace('\n', ' ')
                        print(f"  Rij {idx + 1} ❌ Fout: {error_msg}")
                        continue
            
            print("\n⏳ Herinschakelen van FOREIGN KEY constraints...")
            cursor.execute("EXEC sp_MSforeachtable 'ALTER TABLE ? CHECK CONSTRAINT ALL'")
            conn.commit()
            
            print("\n🎉 Upload voltooid voor alle tabellen!")
            cursor.close()
            conn.close()
            
    except pyodbc.Error as e:
        print(f"❌ Databasefout: {e}")

# Roep de functie aan met je dataWarehouse dictionary
upload_dataframes_to_sql(dataWarehouse)

⏳ Uitschakelen van FOREIGN KEY constraints...

Bezig met uploaden van tabel: fact_sales...
  Rij 1 ✅ Upload voltooid
  Rij 2 ✅ Upload voltooid
  Rij 3 ✅ Upload voltooid
  Rij 4 ✅ Upload voltooid
  Rij 5 ✅ Upload voltooid
  Rij 6 ✅ Upload voltooid
  Rij 7 ✅ Upload voltooid
  Rij 8 ✅ Upload voltooid
  Rij 9 ✅ Upload voltooid
  Rij 10 ✅ Upload voltooid
  Rij 11 ✅ Upload voltooid
  Rij 12 ✅ Upload voltooid
  Rij 13 ✅ Upload voltooid
  Rij 14 ✅ Upload voltooid
  Rij 15 ✅ Upload voltooid
  Rij 16 ✅ Upload voltooid
  Rij 17 ✅ Upload voltooid
  Rij 18 ✅ Upload voltooid
  Rij 19 ✅ Upload voltooid
  Rij 20 ✅ Upload voltooid
  Rij 21 ✅ Upload voltooid
  Rij 22 ✅ Upload voltooid
  Rij 23 ✅ Upload voltooid
  Rij 24 ✅ Upload voltooid
  Rij 25 ✅ Upload voltooid
  Rij 26 ✅ Upload voltooid
  Rij 27 ✅ Upload voltooid
  Rij 28 ✅ Upload voltooid
  Rij 29 ✅ Upload voltooid
  Rij 30 ✅ Upload voltooid
  Rij 31 ✅ Upload voltooid
  Rij 32 ✅ Upload voltooid
  Rij 33 ✅ Upload voltooid
  Rij 34 ✅ Upload voltooid
